## Importando Bibliotecas

In [12]:
import pandas as pd
import os
from io import BytesIO
import requests

StatementMeta(, 6a815165-26f0-48f5-9e14-a4fc5dd00115, 14, Finished, Available, Finished)

## Funções

#### Função para editar dataframe

In [13]:
def edit_sheet(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  
    df.columns = df.columns.str.replace('\n', '_', regex=True)
    df.columns = df.columns.str.replace(' ', '_', regex=True)
    return df

StatementMeta(, 6a815165-26f0-48f5-9e14-a4fc5dd00115, 15, Finished, Available, Finished)

#### Função para salvar dataframe em parquet no lakehouse

In [3]:
def save_to_lake(df, path, filename):
    full_path = f"{path}{filename}"
    df.to_parquet(full_path, index=False)
    print(f"Salvo em: {full_path}")

StatementMeta(, 6a815165-26f0-48f5-9e14-a4fc5dd00115, 5, Finished, Available, Finished)

#### Função para converter data

In [4]:
def convert_iddata(df, col_name):
    return pd.to_datetime(df[col_name], format="%Y%m%d").dt.strftime("%Y%m%d").astype("Int64")

StatementMeta(, 6a815165-26f0-48f5-9e14-a4fc5dd00115, 6, Finished, Available, Finished)

#### Função para salvar em tabela delta

In [5]:
def convert_delta(df, nome):
    # Converter para Spark
    df_spark = spark.createDataFrame(df)

    # Substitui caracteres inválidos nos nomes das colunas
    invalid_chars = [' ', ',', ';', '{', '}', '(', ')', '\n', '\t', '=']
    for char in invalid_chars:
        df_spark = df_spark.toDF(*[col.replace(char, "_") for col in df_spark.columns])

    # Salvar como tabela Delta no Lakehouse
    df_spark.write.mode("overwrite").format("delta").saveAsTable(nome)

StatementMeta(, 6a815165-26f0-48f5-9e14-a4fc5dd00115, 7, Finished, Available, Finished)

## Parâmetros

In [6]:
base1 = {
    "sheet_id": "1cucnW4yVosO5n5BFgwXYv6rVy8yj6NTasM83RTCMOug",
    "gid_receita": "373473243",
    "gid_despesas": "1859279676",
    "gid_PLR": "835809915",
    "encoding": "utf-8",
    "lake_path": "abfss://Hackathon_Emprega_Dados@onelake.dfs.fabric.microsoft.com/lakehouse_vendas.Lakehouse/Files/bronze/",  # Substituir pelo caminho correto no lake
    "dbml_path": "lakehouse://File/bronze/base1.dbml",  # Se quiser salvar dbml no lake
}

base2 = {
    "url": "https://empregadados-my.sharepoint.com/personal/bianca_empregadados_com_br/_layouts/15/download.aspx?share=EZYutqfo5ldNhDw2lMYRxrIBnpPI6c7OTjBBS_F5yz860Q",
    "lake_path": "abfss://Hackathon_Emprega_Dados@onelake.dfs.fabric.microsoft.com/lakehouse_vendas.Lakehouse/Files/bronze/",
    "dbml_path": "lakehouse://path_to_your_modeling/base2.dbml",
    "encoding": "utf-8",
}


StatementMeta(, 6a815165-26f0-48f5-9e14-a4fc5dd00115, 8, Finished, Available, Finished)

## Extração Base 1 - Receita

In [7]:
url_receita = f"https://docs.google.com/spreadsheets/d/{base1['sheet_id']}/export?format=csv&gid={base1['gid_receita']}"
df_Receita = pd.read_csv(url_receita, encoding=base1['encoding'])
df_Receita = edit_sheet(df_Receita)


df_Receita_ConsigCar_estimativa = df_Receita[df_Receita["Faturamento_ConsigCar"] == "Pagseguro"] \
                                .iloc[:, [1,6]].reset_index(drop = True)

for i in range(1,len(df_Receita_ConsigCar_estimativa)):
    if pd.isna(df_Receita_ConsigCar_estimativa.loc[i,'Data']):
        prev_date = pd.to_datetime(df_Receita_ConsigCar_estimativa.loc[i-1, 'Data'], format='%d/%m/%Y')
        next_date = prev_date + pd.DateOffset(months=1)
        if i == 1:  # Caso especial para a primeira data
            df_Receita_ConsigCar_estimativa.loc[i,'Data'] = prev_date.strftime('%d/%m/%Y')
        else:
            df_Receita_ConsigCar_estimativa.loc[i,'Data'] = next_date.strftime('%d/%m/%Y')

############# Verificar isso depois #############
# Gambiarra para corrigir o problema com a primeira data da tabela
# df_Receita_ConsigCar_estimativa.loc[0, 'Data'] = pd.to_datetime('2025-01-01').date()

df_Receita_ConsigCar_estimativa['Data'] = pd.to_datetime(df_Receita_ConsigCar_estimativa['Data'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

# Exclusão de linhas sem dados nas colunas Data e Mês
df_Receita = df_Receita.dropna(subset=["Data", "Mes"])

# Transformação da coluna data
df_Receita['Data'] = pd.to_datetime(df_Receita['Data'], dayfirst = True).dt.date

# Transformação das colunas Mês e Ano para inteiro
df_Receita["Mes"] = df_Receita["Mes"].astype(int)
df_Receita["Ano"] = df_Receita["Ano"].astype(int)

# Separação das colunas da Alucar e Consigcar
df_Receita_Alucar = df_Receita.iloc[:, 0:5] \
                    .reset_index(drop = True)

df_Receita_ConsigCar = df_Receita.iloc[:, [1,6]] \
                        .dropna(subset=["Valor"]) \
                        .reset_index(drop = True)

############# Verificar isso depois #############
# Gambiarra para corrigir o problema com a primeira data da tabela
# df_Receita_ConsigCar.loc[0, 'Data'] = pd.to_datetime('2025-01-01').date()

#     
df_Receita_Alucar_estimativa = df_Receita_Alucar.reset_index(drop = True)

df_Receita_Alucar = df_Receita_Alucar[df_Receita_Alucar["Nome_(Alucar)"] != 'Estimativa']

save_to_lake(df_Receita_Alucar, base1['lake_path'], "b_vendas_clientes_alucar.parquet")
save_to_lake(df_Receita_Alucar_estimativa, base1['lake_path'], "b_vendas_clientes_alucar_estimativa.parquet")
save_to_lake(df_Receita_ConsigCar, base1['lake_path'], "b_receita_pagseguro_consigcar.parquet")
save_to_lake(df_Receita_ConsigCar_estimativa, base1['lake_path'], "b_receita_consigcar_estimativa.parquet")

convert_delta(df_Receita_Alucar, "b_vendas_clientes_alucar")
convert_delta(df_Receita_Alucar_estimativa, "b_vendas_clientes_alucar_estimativa")
convert_delta(df_Receita_ConsigCar,"b_receita_pagseguro_consigcar")
convert_delta(df_Receita_ConsigCar_estimativa,"b_receita_consigcar_estimativa")

StatementMeta(, 6a815165-26f0-48f5-9e14-a4fc5dd00115, 9, Finished, Available, Finished)

Salvo em: abfss://Hackathon_Emprega_Dados@onelake.dfs.fabric.microsoft.com/lakehouse_vendas.Lakehouse/Files/bronze/b_vendas_clientes_alucar.parquet
Salvo em: abfss://Hackathon_Emprega_Dados@onelake.dfs.fabric.microsoft.com/lakehouse_vendas.Lakehouse/Files/bronze/b_vendas_clientes_alucar_estimativa.parquet
Salvo em: abfss://Hackathon_Emprega_Dados@onelake.dfs.fabric.microsoft.com/lakehouse_vendas.Lakehouse/Files/bronze/b_receita_pagseguro_consigcar.parquet
Salvo em: abfss://Hackathon_Emprega_Dados@onelake.dfs.fabric.microsoft.com/lakehouse_vendas.Lakehouse/Files/bronze/b_receita_consigcar_estimativa.parquet


In [11]:
print(df_Receita_Alucar_estimativa)

StatementMeta(, 6a815165-26f0-48f5-9e14-a4fc5dd00115, 13, Finished, Available, Finished)

                    Nome_(Alucar)        Data  Mes   Ano Valor_Receita
0       João Carlos Silva Pereira  2025-01-02    1  2025   R$ 1.432,00
1   Maria Fernanda Souza Oliveira  2025-01-03    1  2025   R$ 4.189,00
2     Marcos Vinícius Silva Costa  2025-01-06    1  2025     R$ 118,00
3        Ana Beatriz Almeida Lima  2025-01-07    1  2025   R$ 2.756,00
4        Luiz Felipe Rocha Santos  2025-01-09    1  2025   R$ 4.631,00
..                            ...         ...  ...   ...           ...
83                     Estimativa  2025-08-01    8  2025  R$ 36.000,00
84                     Estimativa  2025-09-01    9  2025  R$ 36.000,00
85                     Estimativa  2025-10-01   10  2025  R$ 36.000,00
86                     Estimativa  2025-11-01   11  2025  R$ 36.000,00
87                     Estimativa  2025-12-01   12  2025  R$ 36.000,00

[88 rows x 5 columns]


## Extração Base 1 - Despesa

In [8]:

url_despesas = f"https://docs.google.com/spreadsheets/d/{base1['sheet_id']}/export?format=csv&gid={base1['gid_despesas']}"

df_Despesas = pd.read_csv(url_despesas, encoding=base1['encoding'])

df_Despesas.columns = df_Despesas.iloc[0]
df_Despesas = df_Despesas[1:].reset_index(drop=True)

index_total = df_Despesas[df_Despesas['DESPESAS'].str.contains("TOTAL", na=False)].index.min()

df_Despesas_Alucar = df_Despesas.iloc[:index_total].copy().reset_index(drop=True) \
    .melt(id_vars=["DESPESAS"], var_name="Mês", value_name="Valor")

df_Despesas_ConsigCar = df_Despesas.iloc[index_total + 4:-1].copy().reset_index(drop=True) \
    .melt(id_vars=["DESPESAS"], var_name="Mês", value_name="Valor")

# Salvar despesas
save_to_lake(df_Despesas_Alucar, base1['lake_path'], "b_despesas_alucar.parquet")
save_to_lake(df_Despesas_ConsigCar, base1['lake_path'], "b_despesas_consigcar.parquet")

convert_delta(df_Despesas_Alucar,"b_despesas_alucar")
convert_delta(df_Despesas_ConsigCar,"b_despesas_consigcar")

StatementMeta(, 8aefcb5f-7a22-46e9-87bb-93965e6e0ac1, 10, Finished, Available, Finished)

Salvo em: abfss://Hackathon_Emprega_Dados@onelake.dfs.fabric.microsoft.com/lakehouse_vendas.Lakehouse/Files/bronze/b_despesas_alucar.parquet
Salvo em: abfss://Hackathon_Emprega_Dados@onelake.dfs.fabric.microsoft.com/lakehouse_vendas.Lakehouse/Files/bronze/b_despesas_consigcar.parquet


## Metas

In [15]:
url_plr = f"https://docs.google.com/spreadsheets/d/{base1['sheet_id']}/export?format=csv&gid={base1['gid_PLR']}"

df_metas = pd.read_csv(url_plr, index_col=False, header=2, encoding=base1['encoding'])

#Tratamento dos dados relacionados as metas
df_metas.columns = df_metas.columns.map(str)
df_metas = df_metas.loc[:, ~df_metas.columns.str.contains('^Unnamed')]
df_metas = df_metas.iloc[:-1].fillna(0)
df_metas = edit_sheet(df_metas)
df_metas['Mês'] = df_metas['Mês'].astype(int)
df_metas = df_metas.rename(columns={'Meta_1': 'Meta_1_ALUCAR', 'Meta_2': 'Meta_2_ALUCAR', 'Meta_1.1': 'Meta_1_ConsigCar', 'Meta_2.1': 'Meta_2_ConsigCar'})
df_metas = df_metas[['Ano','Mês', 'Meta_1_ALUCAR', 'Meta_2_ALUCAR', 'Meta_1_ConsigCar', 'Meta_2_ConsigCar']]
df_metas['Ano'] = df_metas['Ano'].iloc[0]
df_metas['Data'] = pd.to_datetime({
    'year': df_metas['Ano'].astype(int),
    'month': df_metas['Mês'].astype(int),
    'day': 1
})

save_to_lake(df_metas, base1['lake_path'], "b_metas_plr.parquet")

convert_delta(df_metas,"b_metas_plr")

StatementMeta(, 8aefcb5f-7a22-46e9-87bb-93965e6e0ac1, 17, Finished, Available, Finished)

Salvo em: abfss://Hackathon_Emprega_Dados@onelake.dfs.fabric.microsoft.com/lakehouse_vendas.Lakehouse/Files/bronze/b_metas_plr.parquet


## Extração Base 2

In [16]:

response = requests.get(base2["url"])
df_base2 = pd.read_excel(BytesIO(response.content))

df_base2['Valor parcela'] = df_base2['Valor parcela'].replace({'R\$': '', '\.': '', ',': '.'}, regex=True).astype(float)
df_base2['Data do Pagamento'] = pd.to_datetime(df_base2['Data do Pagamento']).dt.date

save_to_lake(df_base2, base2['lake_path'], "b_vendas_clientes_consigcar.parquet")

convert_delta(df_base2,"b_vendas_clientes_consigcar")

StatementMeta(, 8aefcb5f-7a22-46e9-87bb-93965e6e0ac1, 18, Finished, Available, Finished)

Salvo em: abfss://Hackathon_Emprega_Dados@onelake.dfs.fabric.microsoft.com/lakehouse_vendas.Lakehouse/Files/bronze/b_vendas_clientes_consigcar.parquet


In [5]:
import pandas as pd

# Caminho para o arquivo Excel dentro do Lakehouse (ajuste o nome)
file_path = "/lakehouse/default/Files/DRE_Estimado.xlsx"

# Carrega com pandas
pdf = pd.read_excel(file_path, dtype={"CodGrupo": str})

# Converte para Spark
df = spark.createDataFrame(pdf)

# Salva como tabela Delta no Lakehouse (aba Tables)
df.write.mode("overwrite").saveAsTable("b_dre_estimado")

StatementMeta(, c1295b89-4747-4614-aa18-9d1df48f8b91, 7, Finished, Available, Finished)